## Pisos turisticos Euskadi
Fuente: https://opendata.euskadi.eus/catalogo/-/viviendas-y-habitaciones-de-vivienda-particular-para-uso-turistico-en-euskadi/

In [1]:
import numpy as np
import pandas as pd

In [2]:
airbnbs = pd.read_csv('./data/pisos_turisticos.csv')
airbnbs

,Provincia,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Araba/Álava,20,34,19,17,10,20,35,28,24,10
1,Bizkaia,75,179,242,339,207,150,317,567,538,148
2,Gipuzkoa,198,373,244,302,128,114,156,265,135,39


Fuente: https://www.ine.es/jaxiT3/Datos.htm?t=39364

In [ ]:
ine_pisos_tur = pd.read_csv('./data/viviendas_turisticas_INE.csv', encoding='latin-1')
ine_pisos_tur

https://www.euskadi.eus/contenidos/ds_recursos_turisticos/habitaciones_viviendas_turisti/opendata/viviendas.xlsx

## Precio alquiler Euskadi - Territorios históricos
Fuente:

In [8]:
precio_alq_th = pd.read_csv('./data/precio_alq_territorios_historicos.csv')
precio_alq_th

,Años: 0,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Araba,489.700000,496.412500,521.335000,534.075000,543.567500,550.740000,571.337500,606.615000,594.625000
1,Bizkaia,570.093429,593.445429,611.301429,624.612571,635.615714,645.053143,668.957429,697.343429,693.017143
2,Gipuzkoa,519.531765,571.392059,576.406471,609.690588,620.437353,623.015000,660.228235,680.255294,680.952941


## Precio alquiler - Comarcas
Fuente : https://www.etxebide.euskadi.eus/estadistica/tablas-estadisticas-de-la-estadistica-del-mercado-de-alquiler-emal-2016-2020-hasta-el-2-trimestre-de-2020/webetxe00-content/es/

In [9]:
precio_alq_comarca = pd.read_csv('./data/precio_alq_comarcas.csv')
precio_alq_comarca

,Código comarca,COMARCA,2023,2022,2021,2020,2019,2018,2017,2016,...,2014,2013,2012,2011,2010,2009,2008,2007,2006,Unnamed: 20
0,1100,AÑANA,9.78,9.86,7.84,4.62,5.31,5.49,5.68,5.68,...,6.69,7.43,8.20,30.75,40.88,57.89,64.43,87.15,69.99,NaN
1,1200,LLANADA ALAVESA,17.86,15.24,13.05,9.30,7.77,6.49,5.89,8.46,...,23.46,36.16,48.85,65.68,73.13,73.17,74.48,74.90,53.26,NaN
2,1300,MONTAÑA ALAVESA,8.60,8.08,10.50,7.65,7.66,5.97,5.58,3.24,...,9.79,20.66,28.17,35.80,51.67,56.93,49.52,52.61,48.35,NaN
3,48100,ARRATIA-NERVIÓN,4.90,2.67,3.64,3.15,2.94,2.99,3.33,2.02,...,16.92,19.16,22.64,28.08,34.03,36.86,38.64,42.85,36.66,NaN
4,20200,BAJO BIDASOA,11.73,11.15,13.73,12.95,12.93,11.95,10.28,9.38,...,14.85,12.29,15.54,15.56,14.31,11.65,16.17,18.44,17.05,NaN
5,48200,GRAN BILBAO,9.00,9.14,8.90,9.46,9.63,9.13,8.39,7.24,...,7.39,6.97,7.51,10.76,12.36,16.15,19.99,22.25,22.59,NaN
6,20100,BAJO DEBA,12.23,10.48,8.12,6.71,4.43,3.72,5.37,5.56,...,6.92,8.94,12.67,12.63,21.15,31.07,33.66,32.57,30.84,NaN
7,20300,ALTO DEBA,9.47,10.36,9.90,8.12,6.59,3.35,2.39,2.44,...,8.99,11.58,15.16,29.53,42.14,44.56,45.27,48.42,32.78,NaN
8,20400,DONOSTIALDEA,6.87,7.11,9.40,10.51,11.50,11.32,10.49,8.61,...,11.02,12.36,14.33,16.92,19.64,21.22,23.14,24.97,22.33,NaN
9,48300,DURANGUESADO,12.15,8.71,9.42,10.47,9.28,9.49,9.42,10.26,...,14.99,17.29,22.20,25.36,25.85,29.11,36.27,37.45,37.32,NaN


## Limpieza de CSV con mismo formato que el de arriba que arreglé a mano

In [3]:
import pandas as pd

def limpiar_csv_eustat(ruta_csv):

    df = pd.read_csv(ruta_csv, sep=';', skiprows=1, encoding='latin1')
    
    df.columns = [col.strip().replace('�', 'ó') for col in df.columns]
    
    for col in df.columns[2:]:
        df[col] = df[col].str.replace(',', '.').astype(float)
    
    return df


## Precio compraventa libre
Fuente: https://www.eustat.eus/elementos/tbl0013664_c.html

In [8]:
import pandas as pd

def limpiar_csv_compraventa(ruta_csv, ruta_salida=None):
    
    df = pd.read_csv(ruta_csv, sep=';', skiprows=6, encoding='latin1', header=None)

    df.dropna(axis=1, how='all', inplace=True)

    columnas = ['Trimestre', 
                'Euskadi_Total', 'Euskadi_Nueva', 'Euskadi_Usada',
                'Álava_Total', 'Álava_Nueva', 'Álava_Usada',
                'Bizkaia_Total', 'Bizkaia_Nueva', 'Bizkaia_Usada',
                'Gipuzkoa_Total', 'Gipuzkoa_Nueva', 'Gipuzkoa_Usada']
    df.columns = columnas

    for col in columnas[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

        df.to_csv(ruta_salida, index=False)

    return df

# df_limpio = limpiar_csv_compraventa("precio_compraventa_libre.csv", "compraventa_limpio.csv")


In [ ]:
# limpiar_csv_compraventa('./data/precio_compraventa_libre.csv','./data/precio_cv_limpio.csv')

In [ ]:
'''
precio_cv_limpio = pd.read_csv('./data/precio_cv_limpio.csv')
precio_cv_limpio
'''

,Trimestre,Euskadi_Total,Euskadi_Nueva,Euskadi_Usada,Álava_Total,Álava_Nueva,Álava_Usada,Bizkaia_Total,Bizkaia_Nueva,Bizkaia_Usada,Gipuzkoa_Total,Gipuzkoa_Nueva,Gipuzkoa_Usada
0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IV,266692.0,285388.0,262782.0,236454.0,230058.0,238988.0,249356.0,262106.0,246462.0,303779.0,390071.0,292504.0
2,III,254463.0,285509.0,248547.0,226078.0,242534.0,218616.0,237741.0,281675.0,228817.0,292993.0,371597.0,286166.0
3,II,242732.0,271268.0,238261.0,209811.0,234319.0,200092.0,228797.0,283675.0,220577.0,277867.0,284619.0,277204.0
4,I,257750.0,340939.0,239411.0,222395.0,249302.0,214980.0,248655.0,370352.0,227077.0,286558.0,348786.0,269892.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,IV,222943.0,241777.0,210582.0,190028.0,196678.0,177155.0,227580.0,272224.0,201192.0,237308.0,252025.0,232039.0
62,III,230591.0,246530.0,220026.0,196690.0,210330.0,170995.0,236909.0,281329.0,213681.0,240455.0,234267.0,243584.0
63,II,236399.0,262575.0,223482.0,179903.0,176847.0,181872.0,253922.0,321502.0,224702.0,238932.0,233662.0,241675.0


In [ ]:
# Función para cambiar los NaN de los años por la media de los trimestres y borrar los datos de los trimestres
import pandas as pd

def precio_año_media(df):
    df = df.copy()
    filas_a_conservar = []
    
    i = 0
    while i < len(df) - 4:
        fila = df.iloc[i]
        if str(fila['Trimestre']).isdigit() and pd.isna(fila[1:]).all():
            # Fila con año y NaNs: calcular media de los 4 trimestres siguientes
            medias = df.iloc[i+1:i+5].mean(numeric_only=True)
            fila_actualizada = fila.copy()
            fila_actualizada[1:] = medias.values
            filas_a_conservar.append(fila_actualizada)
            i += 5  # Saltar año + 4 trimestres
        else:
            i += 1  # Saltar fila normal

    # Crear nuevo DataFrame solo con los años resumen
    df_anual = pd.DataFrame(filas_a_conservar).reset_index(drop=True)
    return df_anual

# Aplicarlo a tu DataFrame
df_venta_anual = precio_año_media(precio_cv_limpio)

# Mostrar el resultado
df_venta_anual.head()


,Trimestre,Euskadi_Total,Euskadi_Nueva,Euskadi_Usada,Álava_Total,Álava_Nueva,Álava_Usada,Bizkaia_Total,Bizkaia_Nueva,Bizkaia_Usada,Gipuzkoa_Total,Gipuzkoa_Nueva,Gipuzkoa_Usada
0,2024,255409.25,295776.00,247250.25,223684.50,239053.25,218169.00,241137.25,299452.00,230733.25,290299.25,348768.25,281441.50
1,2023,247215.00,297118.75,238947.00,221106.75,250499.25,216473.25,234635.75,314956.75,221924.75,276812.75,288307.00,274612.75
2,2022,243582.25,288345.25,236742.50,207139.25,221790.75,203713.00,236409.50,308709.75,225551.50,271810.25,288801.50,269382.75
3,2021,246830.75,293447.00,238524.25,202848.75,209676.00,202206.25,241695.50,307465.25,229103.50,271371.75,292210.00,267535.00
4,2020,241276.00,294809.25,230814.50,198174.50,213654.00,194679.75,237037.25,299383.00,224164.75,265791.25,304111.00,257043.50


In [ ]:
''' DF a CSV
ruta_salida = ('./data/precio_cv_anual.csv')
df_venta_anual.to_csv(ruta_salida, index=False)
'''

In [ ]:
import pandas as pd

df = pd.read_csv("./data/precio_cv_anual.csv")

df_transposed = df.set_index("Año").T

totales = df_transposed[df_transposed.index.str.contains("Total")]

totales = totales.round().astype(int)

totales = totales.reset_index().rename(columns={"index": "Categoría"})

totales.to_csv("./data/precio_cv_final.csv", index=False)


## Licencias de vivienda nueva
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-licencias-de-vivienda-nueva-concedidas-en-el-ultimo-quinquenio-x2030-habitantes/

In [20]:
limpiar_csv_eustat('./data/licencias_vivienda_nueva.csv')


,,,,,,,,,,,,,,,,,,,,,,Resumen de datos
Entidad,NaN,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
Araba,NaN,"16,13","13,85","12,31","9,13","7,84","6,51","5,78","7,63","13,98","20,37","30,92","41,87","58,22","66,29","67,98","70,64","72,58","53,49","46,09","44,55","41,18"
Bizkaia,NaN,"9,12","9,03","9,03","9,58","9,61","9,09","8,31","7,40","8,20","8,73","8,70","10,24","14,72","17,28","20,98","25,40","27,76","25,76","29,55","31,34","53,24"
CAE,NaN,"10,35","9,76","9,75","9,57","9,55","8,72","7,92","7,59","9,23","11,10","13,22","16,63","22,52","26,44","29,50","32,78","34,77","29,67","31,00","31,97","32,06"
Gipuzkoa,NaN,"9,67","9,05","9,74","9,76","10,24","9,14","8,27","7,88","8,75","10,76","12,62","15,74","19,33","23,73","26,48","28,26","29,83","25,83","26,92","27,65","28,36"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48915,Ziortza-Bolibar,"2,32","2,27","2,22","0,00","0,00","2,16","4,32","4,35","4,39","8,87","6,77","4,61","4,68","4,77","0,00","0,00","0,00","0,00","42,74","40,90","33,93"
20028,Zizurkil,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","14,51","14,72","54,29","55,04","55,52","40,23","40,97","1,82","1,83","2,19","2,20","3,67","4,74"
01063,Zuia,"20,31","21,15","21,99","12,65","13,01","7,08","7,04","5,34","6,13","8,98","16,82","18,57","20,00","25,30","25,65","24,38","26,95","29,59","57,07","92,02","132,52"
20081,Zumaia,"6,55","3,34","4,99","4,09","3,07","11,93","10,84","22,83","30,95","33,99","25,05","25,34","30,05","39,04","47,30","51,91","60,68","46,53","34,65","30,13","36,57"


In [ ]:
import pandas as pd

def normalizacion:
    with open("./data/licencias_vivienda_nueva.csv", "r", encoding="utf-8") as f:
        lines = f.readlines()


    start_index = None
    for i, line in enumerate(lines):
        if "Detalle por comarcas" in line:
            start_index = i + 1
            break

    comarcas_lines = lines[start_index:]
    comarcas_lines = [line.strip() for line in comarcas_lines if line.strip() != ""]


    comarcas_data = [line.split(";") for line in comarcas_lines]

    df = pd.DataFrame(comarcas_data[1:], columns=comarcas_data[0])

    for col in df.columns[2:]:
        df[col] = df[col].str.replace(",", ".", regex=False)

    df_nombres = pd.read_csv("./data/comarcas_nombres.csv", dtype=str)

    df = df.rename(columns={df.columns[0]: "Código comarca"})
    df = df.merge(df_nombres, on="Código comarca", how="left")

    cols = ["Código comarca", "COMARCA"] + [col for col in df.columns if col not in ["Código comarca", "COMARCA"]]
    df = df[cols]

    return df.to_csv("./data/licencias_final.csv", index=False)



## Suelo urbanizable
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para normalizar índices de tablas que son porcentuales como las 2 de arriba

In [ ]:
import pandas as pd

def limpiar_csv(ruta_csv):

    df = pd.read_csv(ruta_csv, sep=';', skiprows=1, encoding='latin1')
    
    df.columns = [col.strip().replace('�', 'ó') for col in df.columns]
    
    for col in df.columns[2:]:
        df[col] = df[col].str.replace(',', '.').astype(float)
    
    return df.to_csv("./data/suelo_v0.csv", index=False)


In [ ]:
# limpiar_csv('./data/Suelo_urbanizable.csv')

In [42]:
import pandas as pd

def normalizacion():
    with open("./data/suelo_v0.csv", "r", encoding="utf-8") as f:
        lines = f.readlines()


    start_index = None
    for i, line in enumerate(lines):
        if "Detalle por comarcas" in line:
            start_index = i + 1
            break

    comarcas_lines = lines[start_index:]
    comarcas_lines = [line.strip() for line in comarcas_lines if line.strip() != ""]


    comarcas_data = [line.split(";") for line in comarcas_lines]

    df = pd.DataFrame(comarcas_data[1:], columns=comarcas_data[0])
    
    # Utiliza csv de nombres correctos para cambiarlo
    for col in df.columns[2:]:
        df[col] = df[col].str.replace(",", ".", regex=False)

    df_nombres = pd.read_csv("./data/comarcas_nombres.csv", dtype=str)

    df = df.rename(columns={df.columns[0]: "Código comarca"})
    df = df.merge(df_nombres, on="Código comarca", how="left")

    cols = ["Código comarca", "COMARCA"] + [col for col in df.columns if col not in ["Código comarca", "COMARCA"]]
    df = df[cols]

    return df.to_csv("./data/suelo.csv", index=False)



In [49]:
import pandas as pd

def normalizacion2():
    with open("./data/suelo_v0.csv", "r", encoding="utf-8") as f:
        lines = f.readlines()


    start_index = None
    for i, line in enumerate(lines):
        if "Detalle por comarcas" in line:
            start_index = i + 1
            break

    comarcas_lines = lines[start_index:]
    comarcas_lines = [line.strip() for line in comarcas_lines if line.strip() != ""]


    comarcas_data = [line.split(";") for line in comarcas_lines]

    df = pd.DataFrame(comarcas_data[1:], columns=comarcas_data[0])

    for col in df.columns[2:]:
        df[col] = df[col].str.replace(",", ".", regex=False)

    df_nombres = pd.read_csv("./data/comarcas_nombres.csv", dtype=str)

    df = df.rename(columns={df.columns[0]: "Código comarca"})
    df = df.merge(df_nombres, on="Código comarca", how="left")

    cols = ["Código comarca", "COMARCA"] + [col for col in df.columns if col not in ["Código comarca", "COMARCA"]]
    df = df[cols]

    return df.to_csv("./data/suelo_final.csv", index=False)



In [50]:
normalizacion2()

## Solicitudes de vivienda que constan en Etxebide ( habitantes)
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/

In [7]:
sol_th = pd.read_csv('./data/solicitudes_vivienda_th.csv', sep=";")
sol_th

,Entidad,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,Araba,45.48,37.30,36.91,35.33,32.37,29.80,27.56,22.75,21.98,24.04,30.83,28.93,37.92,37.70,45.09,53.05,57.82,59.66,61.13
1,Bizkaia,69.39,52.36,55.16,54.52,49.71,45.26,43.63,37.45,37.59,36.69,42.55,40.66,40.56,39.85,41.92,38.33,37.74,37.07,35.16
2,CAE,62.66,50.71,52.55,51.59,46.32,42.47,40.32,34.10,34.10,33.74,40.56,38.50,40.70,40.76,44.03,43.33,44.08,43.64,41.13
3,Gipuzkoa,59.93,54.29,55.60,54.38,47.32,43.81,40.86,33.89,34.01,33.40,41.74,39.31,42.18,43.62,47.02,47.22,48.41,47.40,42.16


## Viviendas adjudicadas por Etxebide en el ultimo trienio por cada 100 solicitudes inscritas
Fuente:  https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/

## Poblacion total (hab.)
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para normalizar índices de tablas que son porcentuales como las 2 de arriba

## Población extranjera
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para una hipotesis

## Tasa crecimiento vegetativo
Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Necesario para normalizar índices de tablas que son porcentuales como las 2 de arriba

## Mujeres candidatas en elecciones municipales (% sobre total personas candidatas).csv
## Mujeres electas en elecciones municipales (% sobre total personas electas).csv
Fuente: Fuente: https://opendata.euskadi.eus/catalogo/-/indicadores-municipales-de-sostenibilidad-vivienda/  
Hipótesis por determinar todavía
